In [ ]:
class Base:
    def __init__(self):
        self.data=self.Data()

    def Data2(self):
        self.test()
    def test(self):
        print('base')

class New(Base):
    def __init__(self):
        pass
    def Data(self):
        return 22
    def test(self):
        print('parent')
Son=New()
Son.Data2()

In [1]:
import os
import h5py
import tables as tb
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

filetype='dat'
#inputFile="/home/david/TEMP/TestData/testdata openephys/2016-11-04_11-49-24/experiment1_103.raw.kwd"
#inputFile="/data/SWI002/2/SWI002_2016_09_20_12_31/SWI002_2016_09_20_12_31.raw.kwd"

#sampling_rate=kwd['recordings']['0'].attrs['sample_rate']
#f = h5py.File('inputFile','r')
fnameout = inputFile[:-7] + filetype

with tb.open_file(inputFile,"r") as f:
    #data = f['recordings'][str(0)]['data'][:,:]
    print((f.get_node("/recordings/0/data").read().shape))
    print((f.get_node('/recordings/0/')._v_attrs.__getitem__('sample_rate')))
    print(f.root.recordings._v_nchildren)

NameError: name 'inputFile' is not defined

In [60]:
f = h5py.File("/data/Rat105/Experiments/Rat105_2016_12_12_16_11/Rat105_2016_12_12_16_11.kwik",'r')
info=f['recordings']['1']['data'].shape
print((info))

KeyError: "Unable to open object (Object '1' doesn't exist)"

In [4]:
import KWIKphy
from KWIKphy.session import Session
f = "/data/Rat105/Experiments/Rat105_2016_12_12_16_11/Rat105_2016_12_12_16_11.kwik"
session= Session(f)

Features and masks initialized.
Initializing statistics: 100.0%.
Statistics initialized.
Initializing statistics: 100.0%.


In [19]:
print (session.model._metadata)
for shank in session.model._channel_groups:
    session.model._channel_group=shank
    print(shank,session.model._channel_group)

{'excerpt_size': 20000, 'experiment_name': 'Rat105_2016_12_12_16_11.kwik', 'chunk_size': 20000, 'nexcerpts': 50, 'raw_data_files': 'Rat105_2016_12_12_16_11.dat', 'sample_rate': 20000.0, 'connected_component_join_size': 1, 'save_high': 0, 'waveform_filter': True, 'voltage_gain': 10, 'save_low': 0, 'pca_n_waveforms_max': 10000, 'threshold_weak_std_factor': 2.0, 'filter_high': 9500.0, 'excerpt_size_seconds': 1.0, 'n_excerpts': 50, 'chunk_overlap': 300, 'threshold_strong_std_factor': 4.5, 'chunk_size_seconds': 1.0, 'nfeatures_per_channel': 3, 'filter_high_factor': 0.47499999999999998, 'chunk_overlap_seconds': 0.014999999999999999, 'n_channels': 37, 'save_raw': 1, 'nbits': 16, 'waveform_dc_offset': None, 'use_single_threshold': 1, 'filter_butter_order': 3, 'filter_low': 500.0, 'prb_file': 'Rat105_2016_12_12_16_11.prb', 'extract_s_before': 16, 'waveform_scale_factor': None, 'weight_power': 2, 'n_features_per_channel': 3, 'pca_nwaveforms_max': 10000, 'extract_s_after': 16, 'features_contiguou

In [16]:
session.model.__dict__.keys()

dict_keys(['_channel_group', 'kwik_path', '_spikes_per_cluster', '_features', '_metadata', '_opened_files', '_waveform_loader', '_channel_groups', '_kwik', '_clustering_metadata', '_probe', '_channel_order', '_spike_recordings', '_spike_clusters', 'creator', '_features_masks', '_masks', '_traces', '_kwx', '_recordings', '_clustering', '_cluster_metadata', '_channels', '_waveforms', '_recording_offsets', 'name', '_spike_samples', '_waveform_filter', '_clusterings'])

In [ ]:
#To read a kwik file using KWIKphy
from KWIKphy.session import Session
inputFile="/data/SWI002/2/SWI002_2016_09_20_12_52/SWI002_2016_09_20_12_52.kwik"
session= Session(inputFile)


In [ ]:
import h5py

In [ ]:
f = h5py.File("/data/Rat105/Experiments/Rat105_2016_12_05_17_57/Rat105_2016_12_05_17_57.kwx")

In [2]:
experiment="Rat105_2016_12_14_16_53"
animal= experiment[:6]
root="/data/"
%run loadRat_documentation.ipynb
%run plotRat_documentation_1_GeneralBehavior.ipynb
param={
    "goalTime":7,
    "treadmillRange":[0,90],
    "maxTrialDuration":15,
    "interTrialDuration":10,
    "endTrial_frontPos":30,
    "endTrial_backPos":55, 
    "endTrial_minTimeSec":4,
    "binSize":0.25,
}  
data=Data(root,animal,experiment,param,redoPreprocess=True,PrintWarning=True)


Preprocessing behavior data...
No entrance time for trials: [90] ,they will be skipped
No entrance time for trials: [90] ,they will be skipped
No position data for trial #: [ 88.]
140 trials in position file but 138 trials acceptable!
Preprocessing done
you must: %run loadRawSpike_documentation.ipynb for spike data


In [3]:
cameraStartTime=data.cameraStartTime
datfile="/data/Rat105/Experiments/Rat105_2016_12_14_16_53/Rat105_2016_12_14_16_53.dat"
dat=np.fromfile(datfile,dtype=np.int16)
dat=np.reshape(dat,(-1,1),order='C')
dat=np.reshape(dat,(-1,37),order='C')

In [5]:
def Clock2Rate(datfile,chNb,rateTH):
    #datfile="/data/Rat105/Experiments/Rat105_2016_12_14_16_53/Rat105_2016_12_14_16_53.dat"
    dat=np.fromfile(datfile,dtype=np.int16)
    dat=np.reshape(dat,(-1,1),order='C')
    dat=np.reshape(dat,(-1,37),order='C')
    
    clock=dat[:,-chNb].copy()
    del dat
    clockM=np.mean(clock)
    clock[clock<clockM]=0
    clock[clock>=clockM]=1 #clock is a perfect binary signal now
    frameShotTime=np.diff(clock)
    frameShotTime[frameShotTime<0]=0 #a single pulse on the rising edge
    del clock
    frameRateIndex=np.reciprocal(np.diff(np.nonzero(frameShotTime>0.5)[0])/2e4)
    plt.plot(frameRateIndex,'.')
    plt.show()
    frameRateIndex[frameRateIndex>rateTH]=200
    frameRateIndex[frameRateIndex<=rateTH]=25
    timeFrameRate=frameShotTime.copy()
    timeFrameRate[np.nonzero(frameShotTime>0.5)[0][0:-2]]=frameRateIndex.copy()
    timeFrameRate[-1]=timeFrameRate[-2]
    frameType={
        0: np.nonzero(timeFrameRate==25)[0], #slow
        1: np.nonzero(timeFrameRate==200)[0] #fast
        }
    
    index=np.min([frameType[0][0],frameType[1][0]])
    firstFrameType=np.argmin([frameType[0][0],frameType[1][0]])
    frameRate=np.zeros(timeFrameRate.shape)
    
    while True:
        frameRate[index:]=firstFrameType*175+25 #convert to 25 or 200
        firstFrameType = not firstFrameType
        tmpIndex=np.nonzero(frameType[firstFrameType]>index)[0]
        if tmpIndex.size>0:
            index=frameType[firstFrameType][tmpIndex[0]]
        else:
            break
    index=np.max([frameType[0][-1],frameType[1][-1]])
    frameRate[index:]=0
    return frameRate

In [11]:
clock=dat[:,-1].copy()
clockM=np.mean(clock)
clock[clock<clockM]=0
clock[clock>=clockM]=1 #clock is a perfect binary signal now
frameShotTime=np.diff(clock)
frameShotTime[frameShotTime<0]=0 #a single pulse on the rising edge
del clock
frameRateIndex=np.reciprocal(np.diff(np.nonzero(frameShotTime>0.5)[0])/2e4)
frameRateIndex[frameRateIndex>100]=200
frameRateIndex[frameRateIndex<=100]=25
timeFrameRate=frameShotTime.copy()
timeFrameRate[np.nonzero(frameShotTime>0.5)[0][0:-2]]=frameRateIndex.copy()
timeFrameRate[-1]=timeFrameRate[-2]
frameType={
    0: np.nonzero(timeFrameRate==25)[0], #slow
    1: np.nonzero(timeFrameRate==200)[0] #fast
    }

index=np.min([frameType[0][0],frameType[1][0]])
firstFrameType=np.argmin([frameType[0][0],frameType[1][0]])
frameRate=np.zeros(timeFrameRate.shape)

while True:
    frameRate[index:]=firstFrameType*175+25 #convert to 25 or 200
    firstFrameType = not firstFrameType
    tmpIndex=np.nonzero(frameType[firstFrameType]>index)[0]
    if tmpIndex.size>0:
        index=frameType[firstFrameType][tmpIndex[0]]
    else:
        break
index=np.max([frameType[0][-1],frameType[1][-1]])
frameRate[index:]=0

/home/david/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:12: DeprecationWarning: assignment will raise an error in the future, most likely because your index result shape does not match the value array shape. You can use `arr.flat[index] = values` to keep the old behaviour.


In [12]:
cameraStartTime2=frameRate.copy()
cameraStartTime2[cameraStartTime2<100]=0
cameraStartTime2[np.append(0,np.diff(cameraStartTime2))<=0]=0
trialDuration2=np.diff(np.nonzero(cameraStartTime2)).T/2e4
trialDuration1=np.diff(cameraStartTime)
trialDuration1=np.reshape(trialDuration1,trialDuration2.shape)



In [ ]:
trdRate=Clock2Rate(datfile,2,10)
trdStartTime2=trdRate.copy()
trdStartTime2[trdStartTime2<100]=0
trdStartTime2[np.append(0,np.diff(trdStartTime2))<=0]=0
trdDelay2=(np.nonzero(trdStartTime2)[0]-np.nonzero(cameraStartTime2)[0])/2e4

In [28]:
datfile="/data/Rat105/Experiments/Rat105_2016_12_14_16_53/Rat105_2016_12_14_16_53.dat"
f=open(datfile,'rb')
print(np.dtype(np.int16).itemsize)
f.seek(int(60*2e4*37*2),os.SEEK_SET) #np.dtype(np.int16).itemsize
dat0=np.fromfile(f, dtype=np.int16, count=int(60*2e4*37))
dat0=np.reshape(dat0,(-1,1),order='C')
dat0=np.reshape(dat0,(-1,37),order='C')

In [89]:
prmFile='/data/Rat107/Experiments/Rat107_2017_03_20_10_38/Rat107_2017_03_20_10_38.prm'
def a():
    x=1
    def b():
        CWD=os.getcwd()
        os.chdir('/data/Rat107/Experiments/Rat107_2017_03_20_10_38')
        prmName=os.path.basename(prmFile)
        %run -i $prmName
        os.chdir(CWD)
        return locals()
    return b()


In [4]:
dat="/data/Rat107/Experiments/Rat107_2017_03_22_11_35/Rat107_2017_03_22_11_35.dat"
dat=np.fromfile(dat,dtype=np.int16)
dat=np.reshape(dat,(-1,13),order='C')
data=dat[:,6]
del dat

In [251]:
data=pd.read_csv("/data/Rat082/Rat082.tag",delim_whitespace=True,skiprows=range(5))
data

,Sessions,Tag,Speed,Type,Event
0,%Rat082_2016_07_25_14_23,Early-Lesion_DLS,10.0,Good,-
1,%Rat082_2016_07_26_13_20,Early-Lesion_DLS,10.0,Good,-
2,%Rat082_2016_07_27_13_41,Early-Lesion_DLS,10.0,Good,-
3,%Rat082_2016_07_28_14_22,Early-Lesion_DLS,10.0,Good,-
4,%Rat082_2016_07_29_14_31,Early-Lesion_DLS,10.0,Good,-
5,%Rat082_2016_08_01_13_37,Early-Lesion_DLS,10.0,Good,-
6,%Rat082_2016_08_02_12_43,Early-Lesion_DLS,10.0,Good,-
7,%Rat082_2016_08_03_14_29,Early-Lesion_DLS,10.0,Good,-
8,%Rat082_2016_08_04_12_23,Early-Lesion_DLS,10.0,Good,-
9,%Rat082_2016_08_05_12_36,Early-Lesion_DLS,10.0,Good,-


In [108]:
def read_tag_header(root,animal):
    tagPath=os.path.join(root,animal,animal+'.tag')
    L=[]
    out=dict()
    try:
        with open(tagPath,'r') as f:
            for line in f:
                if line[0]=='#':
                    items=line.split(':')
                    out[items[0][1:]]=items[1][:-1]
                else:
                    break
    except:
        return False
    return out
a=read_tag_header('/home','david')
a

{'info': 'test',
 'initialSpeed': '26',
 'name': 'Rat234',
 'option': 'not used',
 'rewardType': 'OK'}

In [27]:
a=datetime.datetime.strptime("2015_02_14","%Y_%m_%d")
b=datetime.datetime.strptime("2015_02_13_15_45","%Y_%m_%d_%H_%M")
a>b

True

In [28]:
'Rat110_2016_05_25_15_65' > 'Rat110_2016_06_00_15_64'

False